In [1]:

import os
import sys
os.environ["SPARK_HOME"] = "/usr/hdp/current/spark2-client"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
sys.path.insert(0, os.environ["PYLIB"] + "/py4j-0.10.4-src.zip")
sys.path.insert(0, os.environ["PYLIB"] + "/pyspark.zip")


## Create SparkContext, SparkSession
from pyspark.sql import SparkSession
from pyspark import SparkContext
sc = SparkContext()

In [2]:

sc

<SparkContext master=local[*] appName=pyspark-shell>

In [3]:
data = sc.textFile('/user/2075B43/Cute_PartB/FlumeData.*')

In [5]:
data.take(10)

[u'txnNo|tDate|customerNo|merchantCity|state|item_category|item_product|item_amount',
 u'00000000|06-27-2018|4004819|Brownsville|Texas|Team Sports|Cheerleading|015.82',
 u'00000000|06-27-2018|4004819|Brownsville|Texas|Water Sports|Whitewater Rafting|086.47',
 u'00000000|06-27-2018|4004819|Brownsville|Texas|Exercise & Fitness|Gym Mats|063.08',
 u'00000000|06-27-2018|4004819|Brownsville|Texas|Exercise & Fitness|Weightlifting Machine Accessories|068.80',
 u'00000000|06-27-2018|4004819|Brownsville|Texas|Team Sports|Lacrosse|092.49',
 u'00000000|06-27-2018|4004819|Brownsville|Texas|Outdoor Recreation|Lawn Games|083.92',
 u'00000001|02-07-2018|4003459|Houston|Texas|Water Sports|Water Tubing|089.28',
 u'00000001|02-07-2018|4003459|Houston|Texas|Water Sports|Surfing|042.38',
 u'00000001|02-07-2018|4003459|Houston|Texas|Team Sports|Cheerleading|062.80']

In [6]:
fea_data = data.map(lambda line : line.split('|'))

In [7]:
fea_data.take(2)

[[u'txnNo',
  u'tDate',
  u'customerNo',
  u'merchantCity',
  u'state',
  u'item_category',
  u'item_product',
  u'item_amount'],
 [u'00000000',
  u'06-27-2018',
  u'4004819',
  u'Brownsville',
  u'Texas',
  u'Team Sports',
  u'Cheerleading',
  u'015.82']]

# total no of records

In [44]:
header = fea_data.first()
fea_data = fea_data.filter(lambda line : line!= header)

In [45]:
print("total records : " ,fea_data.count())

('total records : ', 5476)


# checking for null

In [50]:
FilteredData = fea_data.filter(lambda x: x is not None).filter(lambda x: x != "")

In [51]:
FilteredData.take(2)
print(FilteredData.count())

5476


In [53]:

x = FilteredData.map { case (state, item_product, item_amount) => ((state, item_product), item_amount) }.reduceByKey(_ + _).map{ case (state, item_product, item_amount) => (id, (item_product, item_amount)) }.groupByKey()


SyntaxError: invalid syntax (<ipython-input-53-a04768af85cd>, line 2)

AttributeError: 'PipelinedRDD' object has no attribute 'select'

In [62]:
FilteredData_revenue = FilteredData.groupByKey('state').sum()
FilteredData_revenue.take(10)

Py4JError: An error occurred while calling None.org.apache.spark.api.python.PythonPartitioner. Trace:
py4j.Py4JException: Constructor org.apache.spark.api.python.PythonPartitioner([class java.lang.String, class java.lang.Integer]) does not exist
	at py4j.reflection.ReflectionEngine.getConstructor(ReflectionEngine.java:179)
	at py4j.reflection.ReflectionEngine.getConstructor(ReflectionEngine.java:196)
	at py4j.Gateway.invoke(Gateway.java:237)
	at py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
	at py4j.commands.ConstructorCommand.execute(ConstructorCommand.java:69)
	at py4j.GatewayConnection.run(GatewayConnection.java:214)
	at java.lang.Thread.run(Thread.java:745)

